# Camada Prata - Transformação e Relacionamento

Este notebook executa a transformação dos dados brutos da camada Bronze, realizando:
- Limpeza e tratamento de dados
- Relacionamento entre tabelas
- Criação de dimensões e fatos
- Preparação para análise

## Estrutura da Camada Prata
- `prata/{dataset}/dt={data}/data.parquet`

In [ ]:
# Instalar dependências se necessário
!pip install -q pandas minio pyarrow

In [ ]:
# Executar script de transformação Prata
exec(open('02_prata_transformacao.py').read())

## Visualizar dados transformados

In [ ]:
import pandas as pd
from minio import Minio
import io

# Configurações
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

minio_client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True
)

# Listar arquivos Prata
objects = minio_client.list_objects(BUCKET_NAME, prefix="prata/", recursive=True)
prata_files = list(objects)

print(f"Total de arquivos na camada Prata: {len(prata_files)}")
for obj in prata_files:
    print(f"📁 {obj.object_name} ({obj.size/1024:.2f} KB)")

In [ ]:
# Visualizar dimensão de municípios
objects = list(minio_client.list_objects(BUCKET_NAME, prefix="prata/dim_municipios/", recursive=True))
if objects:
    latest = max(objects, key=lambda x: x.last_modified)
    response = minio_client.get_object(BUCKET_NAME, latest.object_name)
    df_municipios = pd.read_parquet(io.BytesIO(response.read()))
    response.close()
    response.release_conn()
    
    print("Dimensão de Municípios:")
    print(f"Registros: {len(df_municipios)}")
    print(df_municipios.head(10))

In [ ]:
# Visualizar fato BPC
objects = list(minio_client.list_objects(BUCKET_NAME, prefix="prata/fato_bpc/", recursive=True))
if objects:
    latest = max(objects, key=lambda x: x.last_modified)
    response = minio_client.get_object(BUCKET_NAME, latest.object_name)
    df_bpc = pd.read_parquet(io.BytesIO(response.read()))
    response.close()
    response.release_conn()
    
    print("Fato BPC:")
    print(f"Registros: {len(df_bpc)}")
    print(df_bpc.head(10))
    print(f"\nEstatísticas:")
    print(df_bpc.describe())